In [1]:
with open("a1_d3.txt", "r") as text_file:
        data = text_file.read().split('\n')

In [2]:
def preprocessing_data(data):
    processing_data=[]
    for single_data in data:       
        if len(single_data.split("\t"))==2 and single_data.split("\t")[1]!="":
            processing_data.append(single_data.split("\t")) #split data on tab
    return processing_data

In [3]:
values=preprocessing_data(data)

In [4]:
import re
import string
new_data=[]
for i in values:
  i[0]=i[0].lower() #Convert all upper characters to lower characters
  i[0]=re.sub(r'\d+', '', i[0]) #Remove all numbers
  i[0]=i[0].translate(str.maketrans('', '', string.punctuation)) #Remove all punctuation
  new_data.append(i)

In [5]:
#5 fold cross-validation
crossvalidationtestset={}
crossvalidationdataset={}
size=len(values)
for i in range(5):
  crossvalidationtestset[i]=values[(int)(i*size/5):(int)((i+1)*size/5)]
  crossvalidationdataset[i]=values[0:(int)((i)*size/5)]+values[(int)((i+1)*size/5):size]

In [6]:
def make_dataset(values):
  no_items={}
  dataset={}
  #no_items={'0':  ,'1':  }
  #dataset={'word1':{'0':  ,'1':  },'word2':{'0':  ,'1':  }... }
  #no_items has total no. of words in category 0 and category and 1
  #dataset has no. of appearances of word in each category
  for i in range(len(values)):
    no_items.setdefault(values[i][1],0)
    no_items[values[i][1]]+=1
    splitdata=values[i][0].split() #Split line into words
    for j in range(len(splitdata)):
      dataset.setdefault(splitdata[j],{})
      dataset[splitdata[j]].setdefault(values[i][1],0)
      dataset[splitdata[j]][values[i][1]]+=1
  return dataset, no_items

In [7]:
def calc_prob(word,category,dataset,no_of_items):
	#Assign probability value zero if it cannot be calculated
	if word not in dataset or category not in dataset[word]:
		return 0
	return float(dataset[word][category])/no_of_items[category]


In [8]:
# Weighted probability of a word for a category
def weighted_prob(word,category,dataset,no_of_items):
	# basic probability of a word
	basic_prob=calc_prob(word,category,dataset,no_of_items)
	tot=0
	if word in dataset:
		if '0' in dataset[word]:
			tot+=dataset[word]['0']
		if '1' in dataset[word]:
			tot+=dataset[word]['1']
	#default probability of a word is taken as 0.5
	#default appearnaces are also taken as 1
	#weights are taken as total no. of appearnaces
	weight_prob=((1.0*0.5)+(tot*basic_prob))/(1.0+tot)
	return weight_prob


In [9]:
def test_prob(test,category,dataset,no_of_items):
# To get p(test data | category)	
	split_data=test.split() #Split line into words
	p=1
	for i in split_data:
		p*=weighted_prob(i,category,dataset,no_of_items)
	return p

In [10]:
def naive_bayes(test,dataset,no_of_items):
  #NAIVE BAYES FUNCTION
		#p(A|B) = p(B|A) * p(A)/p(B)
		#A :- Probably of being Category(0/1)
		#B :- Test data
		#p(A|B) :- Category given the Test data
		#Ignore p(B) in the denominator since it doesn't change
	results={}
	category=['0','1']
	for i in category:
		# Category Probability
		category_prob=float((no_of_items[i])/(no_of_items['0']+no_of_items['1']))
    # Number of items in category/total number of items

		# p(test data | category)
		test_prob1=test_prob(test,i,dataset,no_of_items)
		results[i]=test_prob1*category_prob
	return results


In [11]:
def compute_tp_tn_fn_fp(y_act,y_pred):
  tp=0 # tp: true positive
  fp=0 # fp: false positive
  fn=0 # fn: false negative
  tn=0 # tn: true negative
  size=len(y_act)
  for i in range(size):
    if((y_act[i]==1)&(y_pred[i]==1)):
        tp+=1
    elif((y_act[i]==0)&(y_pred[i]==0)):
        tn+=1
    elif((y_act[i]==0)&(y_pred[i]==1)):
        fp+=1
    elif((y_act[i]==1)&(y_pred[i]==0)):
        fn+=1
  return tp, tn, fp, fn

In [12]:
import pandas as pd

/home/pranjal/.local/share/virtualenvs/py-classifier-DqgJ-lTA/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [13]:

accuracy={}
precision={}
recall={}
f_score={}
for i in range(5):
  dataset, no_of_items= make_dataset(crossvalidationdataset[i])
  for x in range(len(crossvalidationtestset[i])):
    result=naive_bayes(crossvalidationtestset[i][x][0],dataset,no_of_items)
    if result['1'] > result['0']:
	    crossvalidationtestset[i][x].append(1)
    else:
	    crossvalidationtestset[i][x].append(0)
  y_act=[row[1] for row in crossvalidationtestset[i]]
  y_act=[int(i) for i in y_act]
  y_pred=[row[2] for row in crossvalidationtestset[i]]
  y_pred=[int(i) for i in y_pred]
  y_act1=pd.DataFrame(y_act)
  y_pred1=pd.DataFrame(y_pred)
  tp, tn, fp, fn= compute_tp_tn_fn_fp(y_act,y_pred) 
  accuracy[i]=((tp+tn)*100)/float(tp+tn+fn+fp)
  precision[i]=(tp*100)/float(tp+fp)
  recall[i]=(tp*100)/float(tp+fn)
  f_score[i]=(2*precision[i]*recall[i])/(precision[i] + recall[i])

In [14]:
import math

In [15]:

meanaccuracy=0
for i in range(5):
  meanaccuracy+=accuracy[i]
meanf_score=0
meanaccuracy/=5
for i in range(5):
  meanf_score+=f_score[i]
meanf_score/=500
stddevaccuracy=0
stddevf_score=0
for i in range(5):
  stddevaccuracy+=(accuracy[i]-meanaccuracy)**2
stddevaccuracy/=5
stddevaccuracy=math.sqrt(stddevaccuracy)
for i in range(5):
  stddevf_score+=(f_score[i]/100-meanf_score)**2
stddevf_score/=5
stddevf_score=math.sqrt(stddevf_score)

In [16]:
print("Ans",meanaccuracy,"+-",stddevaccuracy)
print("Ans",meanf_score,"+-",stddevf_score)

Ans 76.9 +- 1.9339079605813714
Ans 0.7738388767626625 +- 0.02527368677944865
